In [2]:
def add(x, y):
    return x + y

def subtract(x, y):
    return x - y

def multiply(x, y):
    return x * y

def divide(x, y):
    if y == 0:
        return "0で割ることはできません"
    return x / y

print("簡単な電卓")
print("1, 足し算")
print("2, 引き算")
print("3, 掛け算")
print("4, 割り算")

choice = input("選択してください(1/2/3/4) :")

num1 = float(input("１つ目の数を入力してください: "))
num2 = float(input("１つ目の数を入力してください: "))

if choice == '1':
    print("結果:", add(num1, num2))
elif choice == '2':
    print("結果:", subtract(num1, num2))
elif choice == '3':
    print("結果:", multiply(num1, num2))
elif choice == '4':
    print("結果:", divide(num1, num2))
else:
    print('無効な選択です')

簡単な電卓
1, 足し算
2, 引き算
3, 掛け算
4, 割り算


ValueError: could not convert string to float: ''